In [1]:
import json
import re

In [16]:
questions = dict()
cal_reasoning = 0
with open('../data/gpt-4o-mini__v4.jsonl') as f:
    for line in f:
        data = json.loads(line)
        ids = data['ids']
        questions[ids] = data
        if 'reasoning' in data:
            cal_reasoning += 1
cal_reasoning

1885

In [17]:
len(questions)

1885

In [18]:
with open('../data/gemini-1.5-flash-scored-gpt-4o-mini__v4.jsonl') as f:
    for line in f:
        data = json.loads(line)
        ids = data['ids']
        if 'score' not in questions[ids]:
            questions[ids]['score'] = []
        questions[ids]['score'].append(data['score'])

with open('../data/gpt-4o-mini-scored-gpt-4o-mini__v4.jsonl') as f:
    for line in f:
        data = json.loads(line)
        ids = data['ids']
        if 'score' not in questions[ids]:
            questions[ids]['score'] = []
        questions[ids]['score'].append(data['score'])

In [19]:
len_sql = 0
for ids, data in questions.items():
    if 'score' in data:
        data['avg'] = sum(data['score']) / 2 # len(data['score'])
    len_sql += len(data['sql'])
len_sql /= len(questions)
len_sql


0.9994694960212201

In [20]:
good_ids = set()
for ids, data in questions.items():
    if 'score' in data and data['avg'] == 1:
        good_ids.add(ids)
        if len(data['sql']) > 1:
            print(ids, data)

In [21]:
len(good_ids)

1141

In [22]:
with open('../data/gpt-4o-mini__v4_good.jsonl', 'w') as f:
    for ids, data in questions.items():
        if ids in good_ids:
            f.write(json.dumps(data) + '\n')

In [9]:
count_result_format = 0
good_result_format = 0
for ids, data in questions.items():
    if 'result format' in data['reasoning'].lower():
        count_result_format += 1
        if '### Result Format' in data['reasoning']:
            good_result_format += 1
            print(ids, data['reasoning'])
            break
count_result_format, good_result_format

e9def022-01c5-4dce-a223-103ae6913cd9 To analyze the Price-to-Earnings (P/E) ratio for the Retail companies (MWG, FRT, PSD) from 2020 to Q3 2024 and compare these ratios to the industry average for each year, we need to:

1. Extract the P/E ratio for the specified companies (MWG, FRT, PSD) from the `financial_ratio` table.
2. Extract the industry average P/E ratio for the Retail industry from the `industry_financial_ratio` table.
3. Combine the results to compare the P/E ratios of the companies with the industry average.

Here is the SQL query to achieve this:

```sql
-- ### P/E Ratio Analysis for Retail Companies (MWG, FRT, PSD) vs Industry Average (2020-Q3 2024)

WITH company_pe_ratios AS (
    SELECT 
        fr.stock_code,
        fr.year,
        fr.quarter,
        fr.data AS pe_ratio,
        ci.industry
    FROM financial_ratio fr
    JOIN company_info ci ON fr.stock_code = ci.stock_code
    WHERE fr.ratio_code = 'PE' -- Price-to-Earnings Ratio
      AND fr.stock_code IN ('MWG',

(2, 1)

In [10]:
for ids, data in questions.items():
  if ids == '74710a9c-1b03-4821-b306-61057c8200cf':
    print('QUESTION ===========')
    print(data['question'])  
    print('TABLES ===========')
    print(data['table'])
    print('ANSWERS ===========')
    print(data['reasoning'])
    print('AVG ===========')
    print(data['avg'])
    break

QUESTION ===========
In Q2 2024, analyze the accounts receivable turnover and days sales outstanding (DSO) for FPT, Vingroup (VIC), and Masan Group (MSN). How do these metrics reflect their collection efficiency, and how might their subsidiaries/investments affect these figures?
TABLES ===========
**Company Info**

| stock_code   | company_name                   | en_company_name              | industry               |
|:-------------|:-------------------------------|:-----------------------------|:-----------------------|
| FPT          | Công ty Cổ phần FPT            | FPT Corporation              | Information Technology |
| MSN          | Công ty Cổ phần Tập đoàn Masan | Masan Group Corporation      | Food and Beverages     |
| VIC          | Tập đoàn Vingroup - Công ty CP | VinGroup Joint Stock Company | Real Estate            |

**category_code_ratio**

| ratio_code   | ratio_name                 |
|:-------------|:---------------------------|
| RTSR         | Receivables To Sal

In [11]:
import random

In [14]:
random.randint(0, 1)

1

In [25]:
import tiktoken

def count_tokens(text: str, model: str = "gpt-4") -> int:
    # Select the tokenizer for the model
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)

err = 0
total_tokens = 0
num = 1
s = 0
u = 0
a = 0
with open('../data/message_deepseek-chat__v0.jsonl') as f:
    for line in f:
        try:
            data = json.loads(line)

            # with open('../data/message_deepseek-chat__v0_good.jsonl', 'a') as f:
            #     json.dump(data, f)
            #     f.write('\n')
            for message in data:
                text = message['content']
                total_tokens += count_tokens(text)
                if message['role'] == 'system':
                    s += count_tokens(text)
                elif message['role'] == 'user':
                    u += count_tokens(text)
                else:
                    a += count_tokens(text)
        except:
            err += 1
            print(num, line)
        num += 1
total_tokens, err, u, s, a

34 [{"role": "system", "content": "\n    You are an expert in financial statement and database management. You will be asked to convert a natural language query into a SQL query. \n    You will have the following database description:\n\n    ### Database Description\n\n    You're given database about the financial statments of top Vietnamese companies, either bank, cooperates and securities.\n\n<overall_description>\n\nAll the data in the financial statments are followed by the regulation of Vietnamese Accounting Standard (VAS). The English translation of the \ncategories are followed by the International Financial Reporting Standards (IFRS).\n\nThere are 3 type of financial statments, based on VAS regulation: bank, non-bank corporation and securities firm (firms that provide stock options and financial instruments).\nAll 3 type of reports are stored in one single table, and there will be a sight different between them.\n\nThe database includes two reporting periods for financial state

(1477765, 8, 528491, 591500, 357774)

In [2]:
def is_npl(question: dict):
    if 'non-per' in question['question'].lower() and 'bdr' not in question['reasoning'].lower():
        return True
    return False

In [7]:
file_path = '../data/gpt-4o-mini__v3.jsonl'
new_file_path = '../data/gpt-4o-mini__v3_2.jsonl'

In [8]:
with open(file_path) as f:
    for line in f:
        data = json.loads(line)
        if not is_npl(data):
            with open(new_file_path, 'a') as f:
                json.dump(data, f)
                f.write('\n')